In [7]:
import pandas as pd
import sqlite3

In [243]:
query = '''
WITH hospitals_tn AS(
    SELECT 
        npi,
        entity_type_code AS entity,
        `provider_organization_name_(legal_business_name)` AS organization,
        SUBSTR(provider_business_practice_location_address_postal_code, 1, 5) AS zipcode,
        taxonomy_code
    FROM npi_data
    WHERE 
        entity_type_code = 2
)
SELECT
    from_npi,
    from_tax.grouping AS from_grouping,
    from_tax.classification AS from_classification,
    from_tax.specialization AS from_specialization,
    to_npi,
    htn.organization AS to_organization,
    to_tax.grouping AS to_grouping,
    to_tax.classification AS to_classification,
    to_tax.specialization AS to_specialization,
    SUM(patient_count) AS total_patient
FROM doc_graph
INNER JOIN hospitals_tn AS htn
ON to_npi = htn.npi
INNER JOIN npi_data AS fro
ON from_npi = fro.npi
LEFT JOIN taxonomy AS from_tax
ON fro.taxonomy_code = from_tax.code
LEFT JOIN taxonomy AS to_tax
ON htn.taxonomy_code = to_tax.code
WHERE htn.zipcode IN (
    SELECT zip
    FROM zipcode
    WHERE cbsa = 34980
)
GROUP BY 
    from_tax.grouping,
    from_tax.specialization,
    from_tax.classification,
    to_npi,
    htn.organization,
    to_tax.grouping,
    to_tax.classification,
    to_tax.specialization
ORDER BY total_patient DESC
'''

In [244]:
with sqlite3.connect('../data/referrals.sqlite') as db: 
    referrals = pd.read_sql(query, db)

In [245]:
referrals

,from_npi,from_grouping,from_classification,from_specialization,to_npi,to_organization,to_grouping,to_classification,to_specialization,total_patient
0,1518032820,Allopathic & Osteopathic Physicians,Pathology,Anatomic Pathology & Clinical Pathology,1124075635,ASSOCIATED PATHOLOGISTS LLC,Laboratories,Clinical Medical Laboratory,None,389861
1,1538559919,Laboratories,Clinical Medical Laboratory,None,1003863580,"ASSOCIATED PATHOLOGISTS, LLC",Allopathic & Osteopathic Physicians,Pathology,Anatomic Pathology & Clinical Pathology,330083
2,1518130996,Allopathic & Osteopathic Physicians,Pathology,Anatomic Pathology & Clinical Pathology,1003863580,"ASSOCIATED PATHOLOGISTS, LLC",Allopathic & Osteopathic Physicians,Pathology,Anatomic Pathology & Clinical Pathology,242187
3,1588840706,Allopathic & Osteopathic Physicians,Pathology,Anatomic Pathology & Clinical Pathology,1235186800,"PATHGROUP LABS, LLC",Laboratories,Clinical Medical Laboratory,None,239242
4,1508862863,Allopathic & Osteopathic Physicians,Radiology,Diagnostic Radiology,1861478489,RADIOLOGY ALLIANCE PC,Allopathic & Osteopathic Physicians,Radiology,Diagnostic Radiology,180474
...,...,...,...,...,...,...,...,...,...,...
24046,1578509998,Suppliers,Durable Medical Equipment & Medical Supplies,None,1366554180,"DIALYSIS ASSOCIATES, LLC",Ambulatory Health Care Facilities,Clinic/Center,End-Stage Renal Disease (ESRD) Treatment,11
24047,1669519849,Suppliers,Pharmacy,Community/Retail Pharmacy,1457303083,TENNESSEE ONCOLOGY PLLC,Allopathic & Osteopathic Physicians,Internal Medicine,Hematology & Oncology,11
24048,1053347344,Suppliers,Durable Medical Equipment & Medical Supplies,Parenteral & Enteral Nutrition,1699720029,NATIONAL HEALTH CORPORATION,Nursing & Custodial Care Facilities,Skilled Nursing Facility,None,11
24049,1053347344,Suppliers,Durable Medical Equipment & Medical Supplies,Parenteral & Enteral Nutrition,1730134610,NHC HEALTHCARE-MCMINNVILLE LLC,Nursing & Custodial Care Facilities,Skilled Nursing Facility,None,11


 AND
        ((`provider_organization_name_(legal_business_name)` LIKE '%MEDICAL CENTER%' AND
        `provider_organization_name_(legal_business_name)` NOT LIKE '%HOSPITALIST%') OR 
        (`provider_organization_name_(legal_business_name)` LIKE '%HOSPITAL%' AND
        `provider_organization_name_(legal_business_name)` NOT LIKE '%HOSPITALIST%'))